In [1]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [ ]:
from data_retrieval import lipade_groundtruth
from transformers import BertTokenizer, FlavaTextModel
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

sourceText = 'blip2'

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
corpus = "lipade_groundtruth"
rawPath = "../results/raw/" + corpus + "/"
distancePath = "../results/distance/" + corpus + "/"

model = FlavaTextModel.from_pretrained("facebook/flava-full")
tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")

_,m,y = lipade_groundtruth.getDataset(mode="similar")

def getEmbedding(txt):
    inputs = tokenizer(text=[txt], return_tensors="pt", padding="max_length", max_length=77)
    return model(**inputs).last_hidden_state.detach().numpy()

embeddings = np.array([getEmbedding(txt) for txt in tqdm(m[1])]).reshape(len(m[1]), -1)

  0%|          | 0/279 [00:00<?, ?it/s]/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/transformers/modeling_utils.py:1593: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
100%|██████████| 279/279 [00:10<00:00, 25.89it/s]


In [4]:
distance = 1 - (cosine_similarity(embeddings, embeddings)+1)/2
distance -= np.diag(distance)

In [ ]:
np.save(rawPath + sourceText + "_text_into_flava.npy", embeddings)
np.save(distancePath + sourceText + "_text_into_flava.npy", distance)